In [24]:
import pandas as pd 
import gensim, logging
import ndcg 
import MAP
from functools import reduce

In [3]:
df = pd.read_csv('datasets_recsys/favorite_stories_books_sample.csv', sep=';', encoding='latin')
df.head()

,user_id,story_id,franchise
0,2092980,10986890,Chronicles of Narnia
1,3219196,12221990,Harry Potter
2,6905423,5511855,Harry Potter
3,1059774,6559818,Harry Potter
4,2190284,6176363,Twilight


In [39]:
# user ids 
list_users = list(df['user_id'].unique())

In [4]:
df_stories = pd.read_csv(open('datasets_recsys/ff_books_dataset.csv'), sep='|', encoding='latin')
df_stories

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,story_title,author,franchise,story_id,story_summary,q_words,date_submit,date_update,chapters,reviews
0,Sweet Silence,100016.0,Harry Potter,1895110.0,Draco meets Ron on the shore of the Great Lake...,725.0,"5/30/2002,1/11/2004,11/8/2003,12/2/2003,5/19/2...",6/5/2004,1.0,2.0
1,Draco Omniscient,100016.0,Harry Potter,1574594.0,Draco's views on the world around him.,1274.0,"5/30/2002,1/11/2004,11/8/2003,12/2/2003,5/19/2...",10/26/2003,1.0,8.0
2,Never Gone Forever,100035.0,Harry Potter,767307.0,*slash* Chapter 3 now up...very short...::is ...,3100.0,"5/9/2002,5/8/2002",7/31/2002,3.0,39.0
3,Shadows,100035.0,Harry Potter,765982.0,AU in a big way. Sev is a vampire in a great ...,3598.0,"5/9/2002,5/8/2002",5/15/2002,3.0,35.0
4,Gwen Potter:Harry\'s Girl,100062.0,Harry Potter,396405.0,Gwen Potter...Harry girl...Dark brown hair to ...,120.0,NaN,9/8/2001,0.0,1.0
5,The Question that I Ask,100064.0,Harry Potter,700112.0,Edited version.About Narcissa and Lucius Malfo...,2885.0,4/3/2002,4/13/2002,2.0,12.0
6,Obliviate,100109.0,Harry Potter,4120660.0,Guns n Handcuffs. They could only hope that hi...,1260.0,7/10/2006,3/8/2008,1.0,1.0
7,Mercury,100111.0,Harry Potter,7023642.0,Written for the PhoenixSong summer 2007 challe...,4147.0,12/3/2002,5/26/2011,1.0,2.0
8,The Secret,100109.0,Harry Potter,1998974.0,"I was sitting on the floor, cradling his body ...",10018.0,7/10/2006,8/5/2004,1.0,28.0
9,The Promise,100111.0,Harry Potter,7023633.0,Written for the PhoenixSong summer 2006 challe...,5123.0,12/3/2002,5/26/2011,1.0,1.0


In [44]:

fav_stories = df.merge(df_stories, on='story_id')
fav_stories = fav_stories[['user_id', 'story_id', 'franchise_x','story_summary']]

# historias favoritas del usuario 20920980
fav_stories = fav_stories.loc[fav_stories['user_id']== 2092980]
#fav_stories

In [6]:
fav_stories_id = []
fav_stories_summaries = []
for x in fav_stories[['story_id', 'story_summary']].values.tolist():
    fav_stories_summaries.append(x[1].split(' '))
    fav_stories_id.append(x[0])


stories_id = []
stories_summaries = []
for y in df_stories[['story_id','story_summary']].values.tolist():
    try:
        stories_summaries.append(y[1].split(' '))
        stories_id.append(y[0])
    except:
        continue
    

In [7]:
model = gensim.models.Word2Vec(stories_summaries, min_count=1, workers=5)
word_vectors = model.wv

In [8]:
# ejemplo similaridad de historia favorita con resumen de la primera historia del dataset 
word_vectors.n_similarity(fav_stories_summaries[0], stories_summaries[0]) # ver como lo hace 

0.83763825169591832

In [9]:
contador = 0

results = []

for x in fav_stories_summaries:
    for y in stories_summaries:
        results.append([stories_id[contador],word_vectors.n_similarity(x, y)])  
        contador +=1
    contador = 0

results_df = pd.DataFrame(results)
results_df.head()

,0,1
0,1895110.0,0.837638
1,1574594.0,0.840369
2,767307.0,0.751952
3,765982.0,0.874748
4,396405.0,0.883614


In [25]:
result = results_df.sort_values(by=1, ascending=False)
result.columns = ['story_id', 'similarity']
top10 = result.head(12)
top10 = top10.merge(df_stories, on='story_id')
top10 = top10[['story_id', 'similarity', 'franchise','story_summary']]
top10 = top10._slice(slice(2,12))
top10

,story_id,similarity,franchise,story_summary
2,1.22947e+07,0.986544,Harry Potter,Neville is used to feel lonely and down all th...
3,8.29684e+06,0.986317,Lord of the Rings,"How does one tell their friends that they, an ..."
4,1.22175e+07,0.986142,Harry Potter,"""I know that it might be hard to hear, but the..."
5,1.01348e+07,0.985448,Outsiders,"Gianna Curtis, not a unique name like Ponyboy ..."
6,8.67416e+06,0.985252,Fault in Our Stars,Augustus taught Hazel the important things in ...
7,7.95921e+06,0.985222,Vampire Diaries,You know you can trust it when Elijah tells yo...
8,1.15751e+07,0.984913,Vampire Diaries,"""Being brave is when you have to do something ..."
9,9.98497e+06,0.984728,Twilight,"Say something, she needed him to speak - she n..."
10,4.00576e+06,0.984434,Twilight,This is the story of Jacob attempting to tell ...
11,8975287.0,0.984352,Hunger Games,Sometimes lying to yourself is the only way to...


In [26]:
fav_stories

,user_id,story_id,franchise_x,story_summary
0,2092980,10986890,Chronicles of Narnia,Growing up is never easy. The Pevensie childre...
1,2092980,6828502,Twilight,"There is so much to say in life, and it is not..."


In [27]:
relevants = []
for x in top10['franchise']:
    if x in list(fav_stories['franchise_x']):
        relevants.append(1)
    else:
        relevants.append(0)

print(relevants)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 0]


In [31]:
# para un usuario,, probar promedio 
ndcg10 = ndcg.ndcg(relevants,10)
precision10 = MAP.precision_at_k(relevants,10)
lista_MAP = [MAP.precision_at_k(relevants,x) for x in range(1, len(relevants))]
mean_average_precision = reduce(lambda x, y: x + y, lista_MAP) / len(lista_MAP)

print('ndcg@10: {}'.format(ndcg10))
print('precision@10: {}'.format(precision10))
print('MAP: {}'.format(mean_average_precision))

ndcg@10: 0.37800210039683896
precision@10: 0.2
MAP: 0.038580246913580245


In [66]:
contador = 0

total_results = []

results = []

for u in list_users:
    fav_stories_summaries = fav_stories.loc[fav_stories['user_id']== u]['story_summary']
    
    for x in fav_stories_summaries:
        
        for y in stories_summaries:
            
            try:
                results.append([stories_id[contador],word_vectors.n_similarity(x, y)])
            
            except:
                continue
            
            contador +=1
            
            if len(results) == 10:
                results = results.sort(key=lambda x: x[1])
                total_results.append([results[0:10]])
                results= list()
                
                print(total_results)
                
        contador = 0
    
    # ver como hacerlo 
    #results_df = pd.DataFrame(results)
    #results_df.head()

KeyboardInterrupt: 